In [3]:
import os 
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.engine.url import URL
import numpy as np
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import boto3
from scipy.stats import pearsonr
from scipy.stats import gmean
import statsmodels.api as sm

In [4]:
import sys
sys.path.append("/Users/sebastianbedoyamazo/Documents/siwa_git/iluma-kb")
import utils_schema as us
from utils_schema import get_otu_and_taxo

In [5]:
us

<module 'utils_schema' from '/Users/sebastianbedoyamazo/Google Drive/Mi unidad/siwa/iluma-kb/utils_schema.py'>

In [6]:
load_dotenv("/Users/sebastianbedoyamazo/Documents/Data-exploration-and-others/.env", override=True)

True

In [7]:
os.environ
sys.path.append("/Users/sebastianbedoyamazo/Documents/Data-exploration-and-others")
import utils_dataexp as u

Reading aws keys from .env successfully!!


In [8]:
s3_client = boto3.client("s3")
s3_resource = boto3.resource("s3")

In [9]:
url = URL.create(
    drivername='redshift+redshift_connector',
    host='iluma-kb-1.cn4ff1ztoyt9.us-east-1.redshift.amazonaws.com',
    port=5439,
    database='dev',
    username='awsuser',
    password='Ilumasiwa1'
)

engine = create_engine(url)
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

redshift_metadata = sa.MetaData(bind=session.bind)

# df performance 42 days

In [101]:
df = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa/siwa2022/performance/performance_df/performance_E345.csv")
df.head()

,Projectid,House,Block 2,Pen,Trt,BW42,BWG36_42,BWG0_42,FCR36_42,FCR0_42
0,E345,1,1,1,2,2676.818182,601.136364,2634.521364,1.888091,1.623173
1,E345,1,1,2,8,2642.727273,539.772727,2602.981061,2.148211,1.715326
2,E345,1,1,3,5,2711.818182,672.181818,2647.285731,1.865702,1.661765
3,E345,1,1,4,3,2688.636364,593.045455,2658.405455,2.059477,1.666568
4,E345,1,1,5,1,2697.826087,568.739130,2655.859130,2.054124,1.612985


In [102]:
df[[i for i in df.columns if "42" in i]].columns

Index(['BW42', 'BWG36_42', 'BWG0_42', 'FCR36_42', 'FCR0_42'], dtype='object')

In [103]:
df = df[['Projectid','House', 'Pen', 'Trt','BW42', 'BWG36_42', 'BWG0_42', 'FCR36_42', 'FCR0_42']]
df.head(3)

,Projectid,House,Pen,Trt,BW42,BWG36_42,BWG0_42,FCR36_42,FCR0_42
0,E345,1,1,2,2676.818182,601.136364,2634.521364,1.888091,1.623173
1,E345,1,2,8,2642.727273,539.772727,2602.981061,2.148211,1.715326
2,E345,1,3,5,2711.818182,672.181818,2647.285731,1.865702,1.661765


In [104]:
df.rename(columns = {'House':'house', 'Pen':'pen', 'Projectid':'projectid'}, inplace = True)
#df_pilot['house'] = df_pilot['house'].replace('[^0-9\.]', '', regex=True)
df['house'] = df['house'].astype(float).astype(int)
df['pen'] = df['pen'].astype(float).astype(int)

# metadata (projectid, house, pen)

In [105]:
query = "SELECT * FROM microbiome as M LEFT JOIN kit as K on M.kitid = K.kitid LEFT JOIN animal as A on M.animalid = A.animalid WHERE projectid = 'E345';"
metadata = pd.read_sql(query, engine)
print(metadata.shape)
print(metadata.columns)
metadata.head(3)

(180, 19)
Index(['sampleid', 'fullsampleid', 'kitid', 'runid', 'animalid',
       'samplelocation', 'alphashannon', 'alphaobserved', 'kitid', 'projectid',
       'age', 'treatment', 'treatmentnumber', 'client', 'animalid',
       'animaltype', 'animalnumber', 'house', 'pen'],
      dtype='object')


,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved,kitid,projectid,age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber,house,pen
0,0110_04I,0110_04I-M,0110,20211214,0110_04,ileum,4.406021,86.0,0110,E345,42.0,Diet6,6,Iluma,0110_04,Broiler,04,2.0,5.0
1,0110_04F,0110_04F-M,0110,20211214,0110_04,feces,3.469237,87.0,0110,E345,42.0,Diet6,6,Iluma,0110_04,Broiler,04,2.0,5.0
2,0110_04C,0110_04C-M,0110,20211214,0110_04,cecum,5.923096,170.0,0110,E345,42.0,Diet6,6,Iluma,0110_04,Broiler,04,2.0,5.0


In [106]:
metadata['pen'] = metadata['pen'].astype(float).astype(int)
metadata['house'] = metadata['house'].astype(float).astype(int)

In [107]:
metadata = metadata[['sampleid', 'fullsampleid',
       'samplelocation', 'projectid', 'house', 'pen']]
metadata

,sampleid,fullsampleid,samplelocation,projectid,house,pen
0,0110_04I,0110_04I-M,ileum,E345,2,5
1,0110_04F,0110_04F-M,feces,E345,2,5
2,0110_04C,0110_04C-M,cecum,E345,2,5
3,0107_05C,0107_05C-M,cecum,E345,2,28
4,0107_05I,0107_05I-M,ileum,E345,2,28
...,...,...,...,...,...,...
175,0114_06C,0114_06C-M,cecum,E345,2,34
176,0114_06I,0114_06I-M,ileum,E345,2,34
177,0114_02C,0114_02C-M,cecum,E345,1,9
178,0114_02I,0114_02I-M,ileum,E345,1,9


# merge metadata with perfomance

In [108]:
metadata_per = metadata.merge(df, how='left', left_on =['projectid', 'house', 'pen'], right_on =['projectid', 'house', 'pen'])
print(metadata_per.shape)
print(metadata_per.columns)
metadata_per.head(3)

(180, 12)
Index(['sampleid', 'fullsampleid', 'samplelocation', 'projectid', 'house',
       'pen', 'Trt', 'BW42', 'BWG36_42', 'BWG0_42', 'FCR36_42', 'FCR0_42'],
      dtype='object')


,sampleid,fullsampleid,samplelocation,projectid,house,pen,Trt,BW42,BWG36_42,BWG0_42,FCR36_42,FCR0_42
0,0110_04I,0110_04I-M,ileum,E345,2,5,6,2630.0,624.869565,2593.509565,1.918313,1.676453
1,0110_04F,0110_04F-M,feces,E345,2,5,6,2630.0,624.869565,2593.509565,1.918313,1.676453
2,0110_04C,0110_04C-M,cecum,E345,2,5,6,2630.0,624.869565,2593.509565,1.918313,1.676453


In [109]:
metadata_per.to_csv("/Users/sebastianbedoyamazo/Documents/siwa/siwa2022/performance/performance_df/pq_obj/metadata_E345.csv", header=True, index=False)

# otu_table and taxonomy for age 42

In [110]:
samples = list(metadata_per['sampleid'])
len(samples)

180

In [111]:
otu_table, taxonomy = us.get_otus_taxo_fromsamples(samples)

Shape otu table (1489, 180)
Shape taxonomy (1489, 8)


In [112]:
otu_table.head(3)

,0105_01C,0105_01F,0105_01I,0105_02C,0105_02F,0105_02I,0105_03C,0105_03F,0105_03I,0105_04C,...,0114_03I,0114_04C,0114_04F,0114_04I,0114_05C,0114_05F,0114_05I,0114_06C,0114_06F,0114_06I
0034f9f05383385c8111cbcfbb92e490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0036a4e11fe7a350076bd7cfb698d458,0.0,0.0,0.0,10.0,0.0,0.0,19.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
003ec3769586bba2e2fcd44397f96010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
otu_table = otu_table.reset_index(drop=False)
otu_table.rename(columns = {'index':'OTU'}, inplace = True)
otu_table.head()

,OTU,0105_01C,0105_01F,0105_01I,0105_02C,0105_02F,0105_02I,0105_03C,0105_03F,0105_03I,...,0114_03I,0114_04C,0114_04F,0114_04I,0114_05C,0114_05F,0114_05I,0114_06C,0114_06F,0114_06I
0,0034f9f05383385c8111cbcfbb92e490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0036a4e11fe7a350076bd7cfb698d458,0.0,0.0,0.0,10.0,0.0,0.0,19.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,003ec3769586bba2e2fcd44397f96010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0167df5430d55fedfbd6dc71f5d2b77f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,017902241ef7cef2be735ccda7b0b98e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
otu_table.to_csv("/Users/sebastianbedoyamazo/Documents/siwa/siwa2022/performance/performance_df/pq_obj/otu_table_E345.csv", header=True, index=False)

In [116]:
taxonomy.rename(columns = {'otu':'OTU'}, inplace = True)
taxonomy.head()

,OTU,species,genus,family,order,tclass,phylum,kingdom
0,37125fc6b7326af0e29ddb765d441528,Fusobacterium_mortiferum,Fusobacterium,Fusobacteriaceae,Fusobacteriales,Fusobacteriia,Fusobacteriota,Bacteria
1,3ac7012c538693bcf363c9d4b6d5e9d5,UNKNOWN,Lactobacillus,Lactobacillaceae,Lactobacillales,Bacilli,Firmicutes,Bacteria
2,3bdb934074500737341f4db660f92c78,Lactobacillus_aviarius,Lactobacillus,Lactobacillaceae,Lactobacillales,Bacilli,Firmicutes,Bacteria
3,5a7cff6a4129c8e828fdb25da2d8c410,Lactobacillus_aviarius,Lactobacillus,Lactobacillaceae,Lactobacillales,Bacilli,Firmicutes,Bacteria
4,c7d19e18132944066131f5f138ba46f2,UNKNOWN,Romboutsia,Peptostreptococcaceae,Peptostreptococcales-Tissierellales,Clostridia,Firmicutes,Bacteria


In [117]:
taxonomy.to_csv("/Users/sebastianbedoyamazo/Documents/siwa/siwa2022/performance/performance_df/pq_obj/taxonomy_E345.csv", header=True, index=False)